In [ ]:
import pandas as pd
import numpy as np

# Example structure of player_data and puck_data
# player_data: ['entityId', 'team', 'position_x', 'position_y', 'timeStamp']
# puck_data: ['position_x', 'position_y', 'timeStamp']

# Function to calculate 2D distance between two points
def calculate_2d_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to identify the closest player to the puck
def find_closest_player_to_puck(puck_position, player_data):
    closest_distance = float('inf')
    closest_player_id = None

    for index, row in player_data.iterrows():
        player_position = (row['position_x'], row['position_y'])
        distance = calculate_2d_distance(player_position, puck_position)

        if distance < closest_distance:
            closest_distance = distance
            closest_player_id = row['entityId']

    return closest_player_id

# Main processing loop for gap control calculation
gap_control_data = []

for time_stamp in puck_data['timeStamp'].unique():
    current_puck_data = puck_data[puck_data['timeStamp'] == time_stamp].iloc[0]
    puck_position = (current_puck_data['position_x'], current_puck_data['position_y'])

    # Check if the puck is in the neutral zone
    if -25 <= puck_position[0] <= 25:
        current_player_data = player_data[player_data['timeStamp'] == time_stamp]
        
        # Find the player closest to the puck
        player_with_puck_id = find_closest_player_to_puck(puck_position, current_player_data)
        player_with_puck_team = current_player_data[current_player_data['entityId'] == player_with_puck_id]['team'].iloc[0]

        # Find closest opposing player
        opposing_players = current_player_data[current_player_data['team'] != player_with_puck_team]
        closest_opponent_id = find_closest_player_to_puck(puck_position, opposing_players)

        if closest_opponent_id is not None:
            closest_opponent_position = (
                opposing_players[opposing_players['entityId'] == closest_opponent_id]['position_x'].iloc[0],
                opposing_players[opposing_players['entityId'] == closest_opponent_id]['position_y'].iloc[0]
            )
            gap_distance = calculate_2d_distance(puck_position, closest_opponent_position)

            gap_control_data.append({
                'timeStamp': time_stamp,
                'puck_position': puck_position,
                'player_with_puck': player_with_puck_id,
                'closest_opponent': closest_opponent_id,
                'gap_distance': gap_distance
            })

# Convert gap_control_data to a DataFrame for easier analysis
gap_control_df = pd.DataFrame(gap_control_data)
print(gap_control_df)
